In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.neighbors import BallTree
import os
import sys
from IPython.display import display, HTML
from IPython.display import clear_output
drive.mount('/content/drive')
api = None

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Change this directory to the location of the shared Spotify Data-X Project on Google Drive
#sharedDirectory = '/content/drive/My Drive/Colab Notebooks/Spotify Data-X Project'
sharedDirectory = '/content/drive/My Drive/Spotify Data-X Project'

os.chdir(sharedDirectory)
sys.path.append(sharedDirectory + "/Utils/")
import searchAPI

In [0]:
spotify = pd.read_csv("Data/spotifyCatalog.csv")
# spotify.head()

In [0]:
spotifyCols = spotify.columns
dataCols = ['year', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo','valence']
outputCols = ["name", "artist_names", "year", "genreArrStr", "id"]
X = spotify[dataCols].astype(np.float16)#.values.astype(np.float16)
# spotify.head(2)

In [0]:
def preProcessTrack(track):
  #remember to multiple release year by ~20, tempo by 10, loudness by 10
  track["year"] *= 20
  track["tempo"] *= 10
  track["loudness"] *= 10
  return track

X = preProcessTrack(X).values

In [0]:
tree = BallTree(X, leaf_size=10)

In [8]:
if api is None: api = searchAPI.SpotifyAPI()

https://accounts.spotify.com/authorize?&client_id=9e5d4d5a6a9748fc9832e7d7c773c91c&response_type=token&redirect_uri=https://elleklinton.com/spotify&scope=playlist-read-private%20playlist-modify-public%20playlist-modify-private%20playlist-read-collaborative%20user-library-read%20user-read-email
Go to the above URL and then paste the URL you were redirected to:
https://elleklinton.com/spotify#access_token=BQDoYMKRSds0Meg8nGxtBLplYZ7DGsno81PUlK9TovG9MMD_bSlzt2_9EcDo4cyojWsWx9TtVuUZvZGuyYYMv0C0eVaH335pwQPC0DwWSQcnaPBYKU5kddtrVzdhkqRd0bjNCPlahmN4wpmREW6ywc2RhDc6jUPSXI-RNjpFOTqIRluOzFejyBJt8UM2FI94Y6a7ymROdf0V9nehQT07Ezm7icqugBfuqublarFY&token_type=Bearer&expires_in=3600


In [9]:
def findNearestTracks():
  searchQuery = input("Please enter the name of the track you would like to search:\n")
  clear_output(wait=False)
  queryResults = api.search(searchQuery, limit=5)
  display(queryResults[["track.name", "artist.name", "year"]])
  trackSelection = input("Please enter the index of the track you would like to select as a seed song:\n(e.g. 0)\n")
  clear_output(wait=False)
  track = queryResults.loc[int(trackSelection), :]
  trackData = track.loc[dataCols]
  trackData = trackData.astype(np.float16)
  trackData = preProcessTrack(trackData)
  clear_output(wait=False)
  k = input("How many similar tracks would you like to search for?\n(e.g. 15)\n")
  dist, idx = tree.query(trackData.values.reshape(1,-1), k=int(k))
  clear_output(wait=False)
  print(f"Found {k} similar tracks to {track['track.name']} with an average distance of {round(np.average(dist), 4)}!!!")
  out = spotify.iloc[idx[0], :][outputCols]
  display(out)
  return track

o = findNearestTracks()

Found 15 similar tracks to Firework with an average distance of 91.7476!!!


,name,artist_names,year,genreArrStr,id
231170,Out The Woods,['We Have Band'],2015,"['electr', 'punk', 'indie', 'dance']",6ikDJbJMPofYE2oJqxYZGK
160823,When You Were Mine,['Lady Antebellum'],2011,"['pop', 'country']",6RP8PPC9nnKw47JbJ5WCkL
341851,Let's not fall in love,['BIGBANG'],2015,['pop'],5QhE3ILw6KUR40fe0w35bb
285094,Chambers,['Cymbals Eat Guitars'],2014,"['pop', 'rock', 'indie']",6lJgmU5hai5e6g06dmea8q
221477,Destrucción masiva,['Bunbury'],2013,"['latin', 'rock']",3TdBwEx1zIMjE1CWVi8to1
312822,Touch The Sky - Disfunktion Remix,"['Paul Oakenfold', 'Matt Goss', 'The Concrete ...",2013,"['electr', 'trance']",6vEBHz95jB8jOU5IW1j6PI
44645,Abhaya,"['Desireless', 'Operation of the Sun']",2015,['disco'],7nJci4GnHexJPzCXO5vIPU
256608,Stand On the Horizon,['Franz Ferdinand'],2013,"['rock', 'garage', 'punk', 'indie', 'dance']",7mexukWpLrCVwAztkM8kQl
339648,Loony Tunes,['Blueboy'],2015,"['pop', 'indie']",04B4Azd6Gc8YtYDDbPNK7U
192604,Fuck You Up,['Telepathe'],2015,"['pop', 'indie']",3idLmx124471xF0zbMHx7J
